# SocraticAI

Instructions:


1. Upload your OpenAI API Key via a txt file labeled 'OpenAI_API_KEY.txt'
2.   Upload your PDF reading that you want to talk to via socratic dialogue
3.   Run all of the cells in this section
4.   Run the code block in the Front End section, then click the Gradio link generated.








In [ ]:
!pip install PyPDF2
!pip install tiktoken
!pip install nest-asyncio
!pip install aiohttp
!pip install asyncinit

In [2]:
from PyPDF2 import PdfReader
import tiktoken
import requests
import aiohttp
import asyncio
import nest_asyncio

In [3]:
f = open("OpenAI_API_KEY.txt", "r")
OPENAI_API_KEY = f.read()
nest_asyncio.apply()

In [4]:
class PDFOutline:
  def __init__(self, path):
    #read PDF
    self.fake_word_rate = 400

    print(self._read_PDF(path))

    big_string = "".join(self._read_PDF(path))
    words = big_string.split(" ")
    self.pages = [" ".join(words[i:min(i+self.fake_word_rate, len(words))]) for i in range(0, len(words), self.fake_word_rate)]

    self.original_word_rate = (len(words) / len(self._read_PDF(path))) 

    #construct summaries in parallel
    asyncio.run(self.parallel_summary(self.pages, 28))
   
    #construct outline
    self.outline = self._outline(self.page_summaries, 1)

  async def call_API(self, session, page, ratio, prev, sub):
      MAX_TOKENS = 4096

      #construct zero-shot summary prompt
      summary_prompt = "You are an expert summarizer. You will be provided a excerpt representing the contents of a current page from a text. You will also be provided the text of the pages directly before and after the excerpt. Summarize the excerpt. You can use the previous and subsequent pages to help contextualize the content on the current page. Please make sure to include all important key phrases in your summary."     
      user_prompt = f"Current page excerpt: {page}\nPrevious page excerpt: {prev}\nSubsequent page excerpt: {sub}"
      messages = [{"role": "system", "content": summary_prompt}, {"role": "user", "content": user_prompt}]
      answer_length = MAX_TOKENS - self._num_tokens_from_string(summary_prompt) - self._num_tokens_from_string(user_prompt) - 50

      #query the model
      headers = {"Authorization": f"Bearer {OPENAI_API_KEY}",
                "content-type": "application/json"}  
      params = {"model": "gpt-3.5-turbo", "messages": messages, "max_tokens": answer_length // ratio}
    
      async with session.post(url="https://api.openai.com/v1/chat/completions", json=params, headers=headers) as response:
        result_data = await response.json()
        try:
          summary = result_data["choices"][0]["message"]["content"]
          return summary
        except Exception as e:
          print(e)
          print(result_data)
          print(self._num_tokens_from_string(user_prompt))
          #print(page)
          #print(prev)
          #print(sub)

  async def parallel_summary(self, pages, ratio):
    async with aiohttp.ClientSession() as session:
        tasks = []
        if len(self.pages) == 1:
          tasks.append(self.call_API(session, self.pages[0], ratio, "Not applicable because this is the first page.", "Not applicable because this is the last page."))
        else:
          tasks.append(self.call_API(session, self.pages[0], ratio, "Not applicable because this is the first page.", self.pages[1]))
          for num, page in enumerate(self.pages[1:-1]):
            tasks.append(self.call_API(session, page, ratio, self.pages[num], self.pages[num+2])) #35 is arbitrary here. I needed to experiment with a couple different ratios to find the idea one
          tasks.append(self.call_API(session, self.pages[-1], ratio, self.pages[-2], "Not applicable because this is the last page."))

        page_summaries = await asyncio.gather(*tasks)
        self.page_summaries = {k: v for k, v in enumerate(page_summaries)}
  
  def _read_PDF(self, path):
    reader = PdfReader(path)
    return [page.extract_text() for page in reader.pages]

  def _num_tokens_from_string(self, string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("p50k_base") #encoding for text-davinci-003
    num_tokens = len(encoding.encode(string))
    return num_tokens

  def _summarize(self, text, ratio, prev, sub):
    #define CRFM API setting arguments
    MODEL = "openai/text-davinci-003"
    MAX_TOKENS = 4096

    #construct zero-shot summary prompt
    summary_prompt = "You are an expert summarizer. You will be provided a excerpt representing the contents of a current page from a text. You will also be provided the text of the pages directly before and after the excerpt. Summarize the excerpt. You can use the previous and subsequent pages to help contextualize the content on the current page. Please make sure to include all important key phrases in your summary."     
    user_prompt = f"Current page excerpt: {text}\nPrevious page excerpt: {prev}\nSubsequent page excerpt: {sub}"
    messages = [{"role": "system", "content": summary_prompt}, {"role": "user", "content": user_prompt}]
    answer_length = MAX_TOKENS - self._num_tokens_from_string(summary_prompt) - self._num_tokens_from_string(user_prompt) - 50

    #query the model
    headers = {"Authorization": f"Bearer {OPENAI_API_KEY}",
               "content-type": "application/json"}  
    params = {"model": "gpt-3.5-turbo", "messages": messages, "max_tokens": answer_length // ratio}

    r = requests.post(url="https://api.openai.com/v1/chat/completions", json=params, headers=headers)
    
    try:
      return r.json()["choices"][0]["message"]["content"]
    except:
      print(r.json())
      print(self._num_tokens_from_string(user_prompt))
  

  def _outline(self, page_summaries, ratio):
    MAX_TOKENS = 4096

    #construct zero-shot outline prompt
    outline_prompt = "You are a world class text outliner. The following text contains page by page summaries from a PDF document. The summaries are provided in the form {[page number]: [summary]}. Write an organized outline (a ,b, c, each with subpoints) for the entire PDF document. Make sure to synthesize the main themes across the pages. Aim for a more compact outline. Don't provide pagae numbers. Get right into the outline–do not make any comments first."
    user_prompt = str(page_summaries)
    messages = [{"role": "system", "content": outline_prompt}, {"role": "user", "content": user_prompt}, {"role": "assistant", "content": "Ok, here is the synthesized outline with page numbers."}]

    answer_length = MAX_TOKENS - self._num_tokens_from_string(outline_prompt) - self._num_tokens_from_string(user_prompt) - 50

    headers = {"Authorization": f"Bearer {OPENAI_API_KEY}",
               "content-type": "application/json"}  
    params = {"model": "gpt-3.5-turbo", "messages": messages, "max_tokens": answer_length // ratio, "temperature": 0.6}

    r = requests.post(url="https://api.openai.com/v1/chat/completions", json=params, headers=headers)
    try:
      return r.json()["choices"][0]["message"]["content"]
    except:
      print(r.json())
      print(self._num_tokens_from_string(user_prompt))
  


In [5]:
class Conversation:
  def __init__(self, title, author, outline):
    self.title = title
    self.author = author
    self.pages = outline.pages
    self.page_summaries = outline.page_summaries
    self.outline = outline.outline
    #initial system prompt
    self.messages = [{"role": "system", "content": "You are a world class AI tutor who helps students understand their readings. The student will tell you the name of the text that they are reading. The system will provide you with a page by page summary of the text. You will engage in a socratic dialogue with the student to help them better understand the text. After the student's question, the system may provide you with a relavent text excerpt, which you should refer to."}]
    #setup context for the reading
    self.add_message("user",f"I am reading {self.title} by {self.author}")
    self.add_message("system", f"page_summaries:\n{str(self.page_summaries)}")
    self.add_message("assistant", "Hi, I am here to help you understand your reading. You can ask me any questions you have about the reading, and I'll do my best to answer your questions!")

  def add_message(self, role, content):
    self.messages.append({"role": role, "content": content})

  def get_reply(self):
    headers = {"Authorization": f"Bearer {OPENAI_API_KEY}",
               "content-type": "application/json"}  
    messages = self.messages

    #try to get the relavent page number given the chat history
    relavent_page_num = self.get_page_num()
    try: 
      page_num = int(relavent_page_num)
    except: 
      page_num = None
      print(relavent_page_num)

    if page_num:
      messages.append({"role": "system", "content": self.pages[page_num]})

    #if we run out of context size, then delete messages from the start
    while True:
      params = {"model": "gpt-3.5-turbo", "messages": messages}
      r = requests.post(url="https://api.openai.com/v1/chat/completions", json=params, headers=headers)
      try:
        reply = r.json()["choices"][0]["message"]["content"]
        return (reply, page_num)
      except:
        self.messsages = self.messages[:4] + self.messages[6:]
        messages = self.messages
        if page_num:
          messages.append({"role": "system", "content": self.pages[page_num]})
      
  
  #GPT-as-backend to determine which page number to reference given the question and given the outline
  def get_page_num(self):
    system_prompt = "You are an AI teaching assistant who helps an AI teacher guide a student through their reading. You will also be provided the entire chat history between the AI teacher and the student. A page-by-page summary of the reading will be including at the beginning of the chat history. Your job is to return an integer representing the most relavant page of the reading given the chat history. You are to only return a single number. No words or other symbols."
    
    while True:
      user_prompt = "Chat history: " + str(self.messages)
      params = {"model": "gpt-3.5-turbo", "messages": [{"role": "system", "content": system_prompt}, 
                                                      {"role": "user", "content": user_prompt}]}
      headers = {"Authorization": f"Bearer {OPENAI_API_KEY}",
                "content-type": "application/json"}
      r = requests.post(url="https://api.openai.com/v1/chat/completions", json=params, headers=headers)

      try:
        page_string = r.json()["choices"][0]["message"]["content"] 
        break
      except:
        print("OH NO")
        self.messages = self.messages[:4] + self.messages[6:]
      
    return int(re.search(r'\d+', page_string).group())

In [ ]:
#test the backend in the notebook, without spinning up Gradio
def run_conversation(path, title, author, outline=None): 
  print("type 'QUIT' to quit out of the chat.\n\n")
  print("starting to create outline...")
  if not outline:
    outline = PDFOutline(path)
  print("done creating outline:")
  conversation = Conversation(title, author, outline)
  print(conversation.outline)
  print(f"\n\nAssistant: {conversation.messages[-1]['content']}")
  while True:
    user_question = input("User: ")
    if user_question == 'QUIT':
      break
    conversation.add_message("user", user_question)
    response, num = conversation.get_reply()
    print(num)
    conversation.add_message("assistant", response)
    print(f"Assistant: {response}")
  return conversation, outline


# Frontend




In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import re

def add_text(state, user_question):
    state['conversationObj'].add_message("user", user_question)
    #try to get the page number related to the question
    response, num = state['conversationObj'].get_reply()
    try:
      real_num = (int(num) * state['outline'].fake_word_rate) / state['outline'].original_word_rate
      state['conversationObj'].add_message("assistant", response)
    except Exception as e:
      print(e)
      print(num)
      state['conversationObj'].add_message("assistant", response)
    #str(conversation.get_page_num(user_question)) + 
    state['convoList'] = state['convoList'] + [(user_question, response + f" [This answer is covered by roughly page {int(real_num)}.]")]

    return state, state['convoList']

def read_PDF(state, file):
  state['outline'] = PDFOutline(file.name)
  state['conversationObj']= Conversation(state['title'], state['author'], state['outline'])
  state['convoList'] = state['convoList']+ [('', state['outline'].outline)]
  state['convoList'] = state['convoList'] + [('', "Hi, I am here to help you understand your reading. You can ask me any questions you have about the reading, and I'll do my best to answer your questions!")]
  return state, state['convoList']

def add_author(state, author_name):
  state['author'] = author_name
  return state, state['convoList']

def add_title(state, title):
  state['author'] = title
  return state, state['convoList']


with gr.Blocks(css="#chatbot .overflow-y-auto{height:500px}") as demo:
    chatbot = gr.Chatbot(elem_id="chatbot")
    state = gr.State({'convoList':[], 'conversationObj': None, 'outline': None, 'author':'','title':''})

    with gr.Column():
      with gr.Row(scale=0.15, min_width=0):
          authorTxt = gr.Textbox(show_label=False, placeholder="Author's Name").style(container=False)
          titleTxt = gr.Textbox(show_label=False, placeholder="Title").style(container=False)
      with gr.Column(scale=0.85):
          btn = gr.UploadButton("Upload a PDF📚")
      with gr.Column(scale=0.85):
          txt = gr.Textbox(show_label=False, placeholder="Start chatting").style(container=False)

    txt.submit(add_text, [state, txt], [state, chatbot])
    authorTxt.submit(add_author, [state, txt], [state, chatbot])
    titleTxt.submit(add_title, [state, txt], [state, chatbot])
    btn.upload(read_PDF, [state, btn], [state, chatbot])

demo.launch(debug = True, share=True)
